In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle as pkl

sns.set()

from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score, train_test_split, KFold, GridSearchCV
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn import metrics

In [2]:
df1 = pd.read_csv("flight_LAX_JFK_data_0601_0831.csv")

In [3]:
df1 = df1[~df1['Price'].str.contains("Info")]
df1['Price'] = df1['Price'].str.replace('$', '')
df1.to_csv("flight_LAX_JFK_data.csv", index=False)

/var/folders/46/pfdkgb65581clhq5m08ccz700000gn/T/ipykernel_9902/2129029691.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df1['Price'] = df1['Price'].str.replace('$', '')


In [4]:
print(f"{df1['From'][0]} => {df1['Destination'][0]} has {df1.shape[0]} flights from June 01 to August 31")

LAX => JFK has 12471 flights from June 01 to August 31


In [5]:
def clean_date(date):
    date = pd.to_datetime(date)
    return date

In [6]:
def convert_to_minutes(timing): 
    hours, minutes = timing.split('h')
    hours = int(hours.strip())
    minutes = int(minutes.strip('m'))
    total_minutes = hours * 60 + minutes
    return total_minutes

In [7]:
df1["Duration"] = df1["Duration"].apply(convert_to_minutes)

In [8]:
df1 = df1.rename(columns={'Price': 'Price(USD)'})
df1 = df1.rename(columns={'Duration': 'Duration(min)'})

In [9]:
df1["Date"] = df1["Date"].apply(clean_date)

In [10]:
df1['Price(USD)'] = df1['Price(USD)'].str.replace(',', '')

In [11]:
df1['Price(USD)'] = df1['Price(USD)'].astype(int)

In [12]:
df1['Duration(min)'] = df1['Duration(min)'].astype(int)

In [20]:
def clean_stops(stops):
    if stops == 'nonstop':
        return 0
    elif stops == '1 stop':
        return 1
    elif stops == '2 stops':
        return 2
    elif stops == '3 stops':
        return 3
    else:
        return np.nan

In [21]:
df1["Stops"] = df1["Stops"].apply(clean_stops)

In [22]:
df1

,Price(USD),Company Name,Stops,Duration(min),Destination,From,Date
0,249,American Airlines,0,334,JFK,LAX,2023-06-01
1,227,American Airlines,1,485,JFK,LAX,2023-06-01
2,286,American Airlines,0,335,JFK,LAX,2023-06-01
3,299,American Airlines,0,335,JFK,LAX,2023-06-01
4,299,American Airlines,0,340,JFK,LAX,2023-06-01
...,...,...,...,...,...,...,...
12466,372,American Airlines,1,622,JFK,LAX,2023-08-31
12467,374,American Airlines,1,629,JFK,LAX,2023-08-31
12468,344,JetBlue,1,648,JFK,LAX,2023-08-31
12469,293,JetBlue,1,696,JFK,LAX,2023-08-31


In [23]:
key = df1[['Price(USD)','Duration(min)','Stops']]
key

,Price(USD),Duration(min),Stops
0,249,334,0
1,227,485,1
2,286,335,0
3,299,335,0
4,299,340,0
...,...,...,...
12466,372,622,1
12467,374,629,1
12468,344,648,1
12469,293,696,1


In [25]:
y = key['Price(USD)']
X = key[['Duration(min)','Stops']]

In [26]:
from sklearn import linear_model
lm = linear_model.LinearRegression()
lm.fit(X, y)

LinearRegression()

In [28]:
lm.predict([[360, 0]])

array([396.40098916])

In [31]:
pkl.dump(lm,open('flight_price_analysis.pkl','wb'))